> This notebook is inspired and influenced by the excellent notebook of 'ExtraTrees StratifiedKFold + Memory Optimization' by Munum Butt, here is the notebook [link](https://www.kaggle.com/munumbutt/extratrees-stratifiedkfold-memory-optimization) 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install scikit-learn-intelex
from sklearnex import patch_sklearn
patch_sklearn()

In [ ]:
import numpy as np
import pandas as pd

from scipy.stats import mode
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

KAGGLE_DIR = r'../input/tabular-playground-series-feb-2022/'
LOCAL_DIR = r''
KAGGLE = True
RS = 69420

In [ ]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [ ]:
%%time
if KAGGLE:
    print(f"{'*'*10} Loading Training Data... {'*'*10}")
    train = pd.read_csv(KAGGLE_DIR+'train.csv', index_col=0).pipe(reduce_mem_usage)
    print(f"{'*'*10} Loading Testing Data... {'*'*10}")
    test = pd.read_csv(KAGGLE_DIR+'test.csv', index_col=0).pipe(reduce_mem_usage)
    sub = pd.read_csv(KAGGLE_DIR+'sample_submission.csv').pipe(reduce_mem_usage)
else:
    print(f"{'*'*10} Loading Training Data... {'*'*10}")
    train = pd.read_csv(LOCAL_DIR+'train.csv', index_col=0).pipe(reduce_mem_usage)
    print(f"{'*'*10} Loading Testing Data... {'*'*10}")
    test = pd.read_csv(LOCAL_DIR+'test.csv', index_col=0).pipe(reduce_mem_usage)
    sub = pd.read_csv(LOCAL_DIR+'sample_submission.csv').pipe(reduce_mem_usage)

In [ ]:
target_encoder = LabelEncoder()
train["target"] = target_encoder.fit_transform(train["target"])

X = train.drop(["target"], axis=1)
y = train["target"]

In [ ]:
y_probs = []
scores = []

folds = StratifiedKFold(n_splits=10, shuffle=True)

estimators = 1300
for fold, (train_id, test_id) in enumerate(folds.split(X, y)):  
    X_train = X.iloc[train_id]
    y_train = y.iloc[train_id]
    X_valid = X.iloc[test_id]
    y_valid = y.iloc[test_id]
    
    model = ExtraTreesClassifier(
        n_estimators=estimators,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    
    valid_pred = model.predict(X_valid)
    valid_score = accuracy_score(y_valid, valid_pred)
    
    print("Fold:", fold + 1, "Accuracy:", valid_score)
    
    scores.append(valid_score)
    
    # Save predictions to later submit the mean values
    #if submission: 
    y_probs.append(model.predict_proba(test))
    
    estimators = estimators + 75

In [ ]:
print("Mean accuracy score:", np.array(scores).mean())

In [ ]:
y_prob = sum(y_probs) / len(y_probs)
# The explanations for these numbers are in AMBROSM's code
y_prob += np.array([0, 0, 0.01, 0.03, 0, 0, 0, 0, 0, 0])
y_pred_tuned = target_encoder.inverse_transform(np.argmax(y_prob, axis=1))
pd.Series(y_pred_tuned, index=test.index).value_counts().sort_index() / len(test) * 100

In [ ]:
sub["target"] = y_pred_tuned
sub.to_csv("submission.csv", index=False)